In [2]:
# Asked ChatGPT for a baseline and this is it
# The parameters are the below along with 70 landscape parameters with 
    #Index(['DateTime', 'lat', 'lon', 'wdir15', 't15', 'ws15', 'pres15', 'wdir150',
    #       't150', 'ws150', 'pres150', 'wdir250', 't250', 'ws250', 'pres250',
    #       'wdir500', 't500', 'ws500', 'pres500', 'f', 'fg', 'gust_factor'],
    #      dtype='object')

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd, numpy as np, tensorflow as tf

In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [4]:
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-25ms-24hr-28-2-24.feather')
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
#df = pd.read_feather('D:\Skóli\lokaverkefni_vel\data\merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-13-3-24-stripped-with-klst.feather')
df = df[df.f < df.fg]
df['gust_factor'] = df.fg / df.f

df_unfolded = df.elevations.apply(pd.Series)

df = pd.concat([df, df_unfolded], axis = 1)

df = df.dropna()
df = df.reset_index(drop = True)

In [5]:
df.iloc[:, -70:] = df.iloc[:, -70:].sub(df.station_elevation, axis = 0)

In [6]:
from sklearn.decomposition import PCA

df_landscape_elevation = df.iloc[:, -70:]

df_landscape_elevation = (df_landscape_elevation - df_landscape_elevation.mean()) / df_landscape_elevation.std()

n_components = 5

pca = PCA(n_components=n_components)
compressed_features = pca.fit_transform(df_landscape_elevation)

compressed_df = pd.DataFrame(data = compressed_features, columns = ['PC' + str(i) for i in range(n_components)])

df  = pd.concat([df, compressed_df], axis = 1)

In [7]:
from math import sqrt, sin, cos, acos, pi

def cornerFromCenterLand(row):
    X, Y, d = row.X, row.Y, row.d
    inlandX, inlandY = 520000, 485000

    len_v1 = sqrt((X-inlandX)**2 + (Y-inlandY)**2)

    v1 = ((X - inlandX)/len_v1, (Y - inlandY)/ len_v1)

    outX, outY = X + cos(d * pi / 180), Y + sin(d * pi / 180)

    len_v2 = sqrt(outX**2 + outY**2)

    v2 = (outX / len_v2, outY / len_v2)

    return acos(np.dot(v1, v2))
    

In [8]:
df['relativeCorner'] = df.apply(cornerFromCenterLand, axis = 1)

In [9]:
df.columns

Index([             'X',              'Y',           'time',          'ws_15',
               'ws_250',         'ws_500',          'wd_15',         'wd_250',
               'wd_500',           'p_15',
       ...
                     66,               67,               68,               69,
                  'PC0',            'PC1',            'PC2',            'PC3',
                  'PC4', 'relativeCorner'],
      dtype='object', length=105)

In [10]:
y = df.gust_factor
X = df[['Ri_01', 'Ri_12', 'X', 'Y', 'station_elevation', 'relativeCorner'] + ['PC' + str(i) for i in range(n_components)]]

# Changing the type of X,y so as to work with Tensorflow
X, y = X.values.astype(np.float32), y.values.astype(np.float32)
scaler = StandardScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [55]:
n_units = 256
n_epochs = 200

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=n_units, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=1, activation='linear')
])

In [56]:
model.compile(optimizer='adam', loss=mean_absolute_percentage_error)
# Train the model
model.fit(X_train, y_train, epochs = n_epochs, batch_size = n_units, validation_data = (X_val, y_val))

Epoch 1/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 125.5194 - val_loss: 58.6101
Epoch 2/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 82.0174 - val_loss: 28.4706
Epoch 3/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 61.6829 - val_loss: 17.0460
Epoch 4/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 47.5365 - val_loss: 14.9070
Epoch 5/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 37.5083 - val_loss: 7.0257
Epoch 6/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.1776 - val_loss: 6.9983
Epoch 7/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 26.1109 - val_loss: 6.5641
Epoch 8/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 22.2372 - val_loss: 5.8694
Epoch 9/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 18.6524 - val_loss: 6.0430
Epoch 10/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 16.9230 - val_loss: 7.2859
Epoch 11/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 15.2880 - val_loss: 6.1169
Epoch 12/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9

In [57]:
# Evaluate the model
mape = model.evaluate(X_test, y_test)
print(f'Model evaluates to: {mape}%')

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5061
Model evaluates to: 5.285275936126709%


In [58]:
model.save(f'./saved_models/nn-{n_units}-units.keras')